In [1]:
import tensorflow as tf
import numpy      as np

import directoryFunctions
import extractFrames
import pathlib
import data
import cv2

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing             import image
from tensorflow.keras.models                    import load_model, Sequential

In [2]:
def getPartialModel(savedModelPath):
    loadedModel = load_model(savedModelPath)

    indexOfGAPLayer = -1
    # Get the index of the last GlobalAveragePooling2D layer starting from the back of the list of layers
    for layer in loadedModel.layers[::-1]:
        layer_type = str(type(layer))
        if layer_type != "<class 'tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D'>":
            indexOfGAPLayer -= 1
        else:
            break

    if indexOfGAPLayer != -1:
        indexAfterGAPLayer = indexOfGAPLayer + 1
        partialModel = Sequential(loadedModel.layers[:indexAfterGAPLayer])
    else:
        # The GlobalAveragePooling2D Layer is the last layer. Thus, we get all the layers (no need to slice the list).
        partialModel = Sequential(loadedModel.layers[:])

    return partialModel

In [3]:
def saveFeatures(model, framesPath, videoFileName, destinationPath):
    def extractFeatures(model, framePath):
        frame = image.load_img(framePath, 
                               target_size   = (299,299), 
                               interpolation = "lanczos") # shape: (299, 299, 3): # of dim: 3
        frame_arr = image.img_to_array(frame)             # pixel values in range [0,255]
        frame_arr = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
        frame_arr = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
        features  = model.predict(frame_arr)              # returns numpy array of shape: (1, 2048): # of dim: 2
        features  = features[0]                           # shape: (2048, ): # of dim: 1
        return features
    
    framesPaths = list(sorted(framesPath.glob("*.jpg")))
    numOfDigits = len(str(len(framesPaths)))
    padding     = f'%0{numOfDigits}d'
    for index, framePath in enumerate(framesPaths):
        sequencePath = pathlib.Path(destinationPath/(videoFileName + "_" + str(padding % index)))
        features = extractFeatures(model, framePath)
        features = np.expand_dims(features, axis = 0)
        print(f'Features from Frame # {index} saved at {sequencePath}')
        np.save(sequencePath, features)

In [4]:
def predictCNN(model, framesPath, classNames):
    def getPredictions(model, framePath):
        frame = image.load_img(framePath, 
                               target_size   = (299,299), 
                               interpolation = "lanczos")   # shape: (299, 299, 3): # of dim: 3
        frame_arr   = image.img_to_array(frame)             # pixel values in range [0,255]
        frame_arr   = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
        frame_arr   = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
        predictions = model.predict(frame_arr)
        predictions = predictions[0]
        return predictions
    
    results = {}
    framesPaths = list(sorted(framesPath.glob("*.jpg")))
    for index, framePath in enumerate(framesPaths):
        predictions    = getPredictions(model, framePath)
        topindex       = predictions.argsort()[-1:][0]
        predictedLabel = classNames[topindex]
        percentage     = predictions[topindex] * 100
        results[index] = [predictedLabel, percentage]
    return results

In [5]:
def predictRNN(model, sequencesPath, classNames):
    def getPredictions(model, sequencePath):
        sequence      = np.load(sequencePath)
        sequence      = np.expand_dims(sequence, axis = 0)
        predictions   = model.predict(sequence)
        predictions   = predictions[0]
        return predictions
    
    results = {}
    sequencesPaths = list(sorted(sequencesPath.glob("*.npy")))
    for index, sequencePath in enumerate(sequencesPaths):
        predictions    = getPredictions(model, sequencePath)
        print(f'Prediction: {index}: {predictions}')
        topindex       = predictions.argsort()[-1:][0]
        predictedLabel = classNames[topindex]
        percentage     = predictions[topindex] * 100
        results[index] = [predictedLabel, percentage]
    return results

In [6]:
def createLabeledVideo(videoFilePath, videoFileName, outputPath, cnnPredictions, rnnPredictions):
    outputFilePath = str(outputPath/f'{videoFileName}_labeled.avi')
    vidCap = cv2.VideoCapture(videoFilePath) # Opens a video file for video capturing
    width  = int(vidCap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vidCap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = int(vidCap.get(cv2.CAP_PROP_FOURCC)) # *'XVID'
#     fps    = int(vidCap.get(cv2.CAP_PROP_FPS))
    fps    = 8
    
    videoWriter = cv2.VideoWriter(outputFilePath, fourcc, fps, (width, height))
    tempDirectory = pathlib.Path(r"./temp")
    framesPath = tempDirectory/'Frames'
    frames = sorted(framesPath.glob("*"))
    for index, frame in enumerate(frames):
        frame = cv2.imread(str(frame))
        cv2.putText(frame, f'CNN: {cnnPredictions[index][0]} - {round(cnnPredictions[index][1], 2)}%', 
                    (1, 14), cv2.FONT_HERSHEY_DUPLEX , 0.45, (255, 100, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, f'RNN: {rnnPredictions[index][0]} - {round(rnnPredictions[index][1], 2)}%', 
                    (1, 30), cv2.FONT_HERSHEY_DUPLEX, 0.45, (255, 0, 100), 1, cv2.LINE_AA)
        videoWriter.write(frame)
    videoWriter.release()

In [7]:
def main():
    dataObj    = data.Data()
    classNames = dataObj.classes
    
    tempDirectory = pathlib.Path(r"./temp")
    directoryFunctions.removeDirectory(tempDirectory)
    directoryFunctions.createDirectory(tempDirectory)
    
    videoFilePath   = r"./UCF-101/YoYo/v_YoYo_g02_c04.avi"
    videoFileName   = str(pathlib.Path(videoFilePath).stem)
    
    framesPath = tempDirectory/'Frames'
    directoryFunctions.createDirectory(framesPath)
    extractFrames.extractAllFrames(videoFilePath, videoFileName, str(framesPath))
    
    savedCNNModelPath = r"./Callbacks/CNN/101/ModelCheckpoint/1589239195_CNN_009_4.05.h5" # you can insert the path to a saved model here
    partialCNNModel   = getPartialModel(savedCNNModelPath)
    
    sequencesPath = tempDirectory/'Sequences'
    directoryFunctions.createDirectory(sequencesPath)
    saveFeatures(partialCNNModel, framesPath, videoFileName, sequencesPath)
    
#     savedCNNModel  = load_model(savedCNNModelPath)
#     cnnPredictions = predictCNN(savedCNNModel, framesPath, classNames)
#     print(type(cnnPredictions))
#     print("cnnPredictions")
#     for key, value in cnnPredictions:
#         print(key, value)

    savedRNNModelPath = r"./Callbacks/RNN/101/ModelCheckpoint/1589397436_RNN_045_3.92.h5" # you can insert the path to a saved model here
    savedRNNModel     = load_model(savedRNNModelPath)
    rnnPredictions    = predictRNN(savedRNNModel, sequencesPath, classNames)
#     print("rnnPredictions")
    for key, value in rnnPredictions.items():
        print(key, value)
    
#     createLabeledVideo(videoFilePath, videoFileName, tempDirectory, cnnPredictions, rnnPredictions)

In [8]:
main()

Creating... temp/Frames/v_YoYo_g02_c04_000.jpg 0
Creating... temp/Frames/v_YoYo_g02_c04_001.jpg 1
Creating... temp/Frames/v_YoYo_g02_c04_002.jpg 2
Creating... temp/Frames/v_YoYo_g02_c04_003.jpg 3
Creating... temp/Frames/v_YoYo_g02_c04_004.jpg 4
Creating... temp/Frames/v_YoYo_g02_c04_005.jpg 5
Creating... temp/Frames/v_YoYo_g02_c04_006.jpg 6
Creating... temp/Frames/v_YoYo_g02_c04_007.jpg 7
Creating... temp/Frames/v_YoYo_g02_c04_008.jpg 8
Creating... temp/Frames/v_YoYo_g02_c04_009.jpg 9
Creating... temp/Frames/v_YoYo_g02_c04_010.jpg 10
Creating... temp/Frames/v_YoYo_g02_c04_011.jpg 11
Creating... temp/Frames/v_YoYo_g02_c04_012.jpg 12
Creating... temp/Frames/v_YoYo_g02_c04_013.jpg 13
Creating... temp/Frames/v_YoYo_g02_c04_014.jpg 14
Creating... temp/Frames/v_YoYo_g02_c04_015.jpg 15
Creating... temp/Frames/v_YoYo_g02_c04_016.jpg 16
Creating... temp/Frames/v_YoYo_g02_c04_017.jpg 17
Creating... temp/Frames/v_YoYo_g02_c04_018.jpg 18
Creating... temp/Frames/v_YoYo_g02_c04_019.jpg 19
Creating..